<h1 style="line-height:2em;">
PANDAPROSUMER EXAMPLE:<br>
INTERNAL COMBUSTION ENGINE COMBINED HEAT AND POWER (ICE CHP)
</h1>

Example prepared by:
--------------------
    Izak Oberčkal Pluško, Marko Keber, Katja Klinar, Tine Seljak*, Andrej Kitanovski

    Faculty of Mechanical Engineering, University of Ljubljana, Slovenia
    
    *Contact: tine.seljak@fs.uni-lj.si

DESCRIPTION:
--------------------
This tutorial describes how to create a single ICE CHP element in *pandaprosumer* and connect it to a single consumer. The user can choose the size of the ICE CHP unit, fuel type and altitude at which the ICE CHP unit is installed, while the time-dependent data (chosen cycle, air intake temperature, demand) is read from a an Excel file and is stored in a pandas dataframe.

![network](figures/ice_chp_demand_1.png)

Glossary:
---------
- Network: a configuration of connected energy sources and energy consumers
- Element: a single energy source or a single energy consumer
- Container: a pandaprosumer data structure that contains data of an individual element; each element must have its container 
- Controller: the logic of an element that defines its behaviour and its limits
- General controller: the first controller in the network that interacts with controllers of all other elements; this controller also manages external data
- Map / mapping: a connection between two elements; contains information about the what is exchanged between the elements

Network design philosophy:
--------------------------
In pandaprosumer, a system's component is represented by a network element. Each element is assigned a *container* and its own *element controller*. A container is a structure that contains the component's configuration data (static input data), which can include information that will not change in the analysis such as size, etc. The behaviour of an element is governed by its *controller*. Connections between elements are defined by *mappings*, which couple output parameters of one controller to the input parameter of a controller of a connected element. 
The network is managed by a *general controller* called *ConstProfileController*. This controller is connected to all element controllers and manages time-dependent input data from external sources (e.g. Excel file). For each time step it distributes the time-dependent input data to relevant element controllers. 

CREATING A NETWORK:
--------------------

If we are not in the pandaprosumer parent directory, we should add it to the path so that the program knows where to find the necessary functions:

In [ ]:
import sys
import os

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)

1 - Input data:
---------------

First, let's import libraries required for data management.

In [ ]:
import pandas as pd
from pandapower.timeseries.data_sources.frame_data import DFData

Next, we need to define properties of the ICE CHP element (size, fuel type, altitude, name), which are treated as static input data, i.e. characteristics that don't change during an analysis. Size is the nominal maximum electrical power (unit: [kW]), the fuel type is chosen from the JSON file that contains the fuel map located in *library/chp_maps/fuel_maps.json*. We will use natural gas, which in pandaprosumer is denoted by "ng". The altitude is the altitude above sea level (unit: [m]) of the ICE CHP installation. In examples with more than one ICE CHP instance, we can give each one its own name to easily distinguish between them:

In [ ]:
name = 'example_chp'
size_kw = 500
fuel = 'ng'
altitude_m = 0

We define the period of the analysis by setting its start and end, which have the form "YYYY-MM-DD HH:MM:SS". The time resolution is given in [s]. 

In [ ]:
start = '2020-01-01 00:00:00' 
end = '2020-01-02 00:00:00'
time_resolution_s = 900 # 15 min 
frequency = '15min'

Now we import our time-dependent data and transform it into an appropriate DFData object. DFData is a pandaprosumer object that stores all data of an individual element. The DFData object is limited by the duration of the analysis defined above.

In [ ]:
time_series_data = pd.read_excel('data/input_chp.xlsx') 

duration = pd.date_range(start, end, freq=frequency, tz='utc')                              
time_series_data.index = duration
time_series_input = DFData(time_series_data)

print(time_series_data.head())

2 - Creating elements of the network:
--------------------------------------

In this example, the network is made up of two elements: an energy source and an energy consumer. The source is represented by a single *ICE CHP* element and the consumer is modelled by a single *heat demand* element. 

![elements](figures/ice_chp_demand_2.png)

First we define an empty prosumer *container* object. Each element of the network has its own container, which is later filled with data and results.

In [ ]:
from pandaprosumer2.create import create_empty_prosumer_container

ice_chp_prosumer = create_empty_prosumer_container()

Then we define the period of the analysis using time data for the analysis given above in Section 1.

In [ ]:
from pandaprosumer2.create import create_period

period = create_period(ice_chp_prosumer, time_resolution_s, start, end, 'utc', 'default')

2.1. General element:

In a pandaprosumer network, the first element is a general controller (*Const Profile controller*). It reads time-dependent input data (*input_params*) and sends it to other elements of the network (*output_params*). The element's data is stored in the *ConstProfileControllerData* class. The controller (*ConstProfileController*) for this element is created with the  *create_controlled_const_profile* function. At this point, we pass to the general controller element the previously created prosumer container, titles of data columns (*input_params*) in the input file (Excel file, in this case) and the coresponding names of output columns (*output_params*), the period of the analysis and the time-dependent data in the DFData object.

In [ ]:
from pandaprosumer2.create_controlled import create_controlled_const_profile

input_params = ['q_demand_kw', 'cycle', 't_intake_k']
output_params = ['q_demand_cp_kw', 'cycle_cp', 't_intake_cp_k']

cp_index = create_controlled_const_profile(
    ice_chp_prosumer, input_params, output_params, period, time_series_input)

2.2. ICE CHP element:

We define the ICE CHP element to which we pass the prosumer container and the data that defines the ICE CHP instance, i.e. size, fuel type, altitude, and its name.

In [ ]:
from pandaprosumer2.create_controlled import create_controlled_ice_chp

ice_chp_index = create_controlled_ice_chp(ice_chp_prosumer, size_kw, fuel, altitude_m, name)

2.3. Heat demand element:

Finally, we create the consumer, which is simulated with the heat demand element. We pass the prosumer container to it. The auxiliary scaling parameter allows us to easily adjust the demanded heat by simply scaling it.

In [ ]:
from pandaprosumer2.create_controlled import create_controlled_heat_demand

heat_demand_index = create_controlled_heat_demand(ice_chp_prosumer, scaling=1.0)

3 - Creating connections (mappings) between controllers:
---------------------------------------------------------

![connections](figures/ice_chp_demand_3.png)

For each controller we define how it is connected to other controllers. In this case we use *Generic Mapping*. The main parameter for the map is the flow of thermal energy (*p_th_out_kw*): the output energy flow of one element is linked with the input energy flow of the connected element. 

In [ ]:
from pandaprosumer2.mapping import GenericMapping

3.1. Connection GENERAL CONTROLLER ---> ICE CHP:

The general controller (*initiator_column*) instructs the ICE CHP controller (*responder_column*) what value to use for *cycle* and *t_intake_k*. 

In [ ]:
GenericMapping(
    ice_chp_prosumer,                                                   
    initiator_id=cp_index,                                           
    initiator_column="cycle_cp",                              
    responder_id=ice_chp_index,                                                
    responder_column="cycle"
);  

In [ ]:
GenericMapping(
    ice_chp_prosumer,                                                   
    initiator_id=cp_index,                                           
    initiator_column="t_intake_cp_k",                              
    responder_id=ice_chp_index,                                                
    responder_column="t_intake_k"
);  

3.2. Connection ICE CHP ---> HEAT DEMAND (consumer):

The ICE CHP controller (*initiator_column*) informs the heat demand controller (*responder_column*) of how much thermal power (*p_th_out_kw*) it can supply at each time step. 

In [ ]:
GenericMapping(
    ice_chp_prosumer,                                              
    initiator_id=ice_chp_index,                                             
    initiator_column="p_th_out_kw",
    responder_id=heat_demand_index,                                           
    responder_column="q_received_kw"
);

3.3. Connection GENERAL CONTROLLER ---> HEAT DEMAND (consumer):

The general controller (*initiator_column*) sends the heat demand controller (*responder_column*) information about the actual demand (*q_demand_kw*) at each time step.

In [ ]:
GenericMapping(
    ice_chp_prosumer,                                                  
    initiator_id=cp_index,                                               
    initiator_column="q_demand_cp_kw",
    responder_id=heat_demand_index,                                                
    responder_column="q_demand_kw"
);

RUNNING THE ANALYSIS:
-----------------------

We can now run the analysis with the input data defined above. 

In [ ]:
ice_chp_prosumer.period

In [ ]:
from pandaprosumer2.run_time_series import run_timeseries

run_timeseries(ice_chp_prosumer, period, True);

PRINTING AND PLOTTING RESULTS:
----------------------------------

First, we plot the evolution of the demand from the Excel file.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
time_series_data.plot(y='q_demand_kw');
plt.show()

In [ ]:
print(ice_chp_prosumer.heat_demand)

We then list the available results for the ICE CHP element. Results are stored in the form of time series, which can be plotted on a graph. The *.time_series* command lists all input and output dataframes. 

In [ ]:
print(ice_chp_prosumer.time_series)

Here *.data_source* lists all available dataframes that we then specify with the *index* key in *.loc[]*.

In [ ]:
print(ice_chp_prosumer.time_series.data_source)

Before plotting we have to look at the resulting dataframe to see which quantity (column) we would like to plot. The *index* key can be an integer or the name of a specific component we want to look at.

In [ ]:
print(ice_chp_prosumer.time_series.data_source.iloc[0].df.head())

Now we can plot the evolution of the electrical power that the ICE CHP generates based on the demand. 

In [ ]:
ice_chp_prosumer.time_series.data_source.loc[0].df.p_el_out_kw.plot()
plt.show()

We can also plot other results, for example the total CO2-equivalent emissions.

In [ ]:
ice_chp_prosumer.time_series.data_source.loc[0].df.acc_co2_equiv_kg.plot()
plt.show()

ACKNOWLEDGEMENTS:
-----------------
The authors would like to thank Pratikshya Mohanty and Odile Capron from the Fraunhofer Institute for the help in preparing this tutorial, especially the diagrams. Special thanks also to Pawel Lytaev and colleagues from the University of Kassel for their code reviews and suggestions during the development of the models. Support from the Senergy Nets project, funded by the European Union under the Horizon Europe program (Grant Agreement No. 101075731) is gratefully acknowledged.